## Paso 0. Library import

In [ ]:
!pip install flopy

In [ ]:
# Temporal, para agregar las rutas correctas
%run ../../src/xmf6/ruta_temporal.py
if not(src_path in sys.path[0]):
    sys.path.insert(0, os.path.abspath(a_path + src_path)) 

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import flopy
from flopy.plot.styles import styles
import xmf6
# Definición de parámetros de decaimiento y sorción
from sorption_decay import *


<img src="../figures/flow_mf6.png">

* El modelo de malla consiste de 1 capa, 120 columnas y 1 renglón.
* La longitud del renglón es de 12 [cm].
* La longitud de la columna es 0.1 [cm].
* Con la información anterior se calcula el ancho del renglón, DELC, y de las columnas, DELR, que ambos casos debe ser 0.1 $cm$.
* La parte superior (TOP) de la celda es 1.0 [cm] y la parte inferior (BOTTOM) es cero.
* La longitud de la capa es igual a 1.0 [cm], valor que se calcula de |TOP - BOTTOM|.

**<font color='Green'> [1] Konikow, L. F., Goode, D. J., & Hornberger, G. (1996, January 1). A three-dimensional method-of-characteristics solute-transport model (MOC3D). Water-Resources Investigations Report 96-4267.</font>** https://doi.org/10.3133/wri964267

**<font color='Green'> [2] Eliezer J. Wexler, (1992).
Analytical solutions for one-, two-, and three-dimensional solute transport in ground-water systems with uniform flow. U.S. Geological Survey Techniques of Water-Resources Investigations, Book 3, Chapter B7, 190 p.</font>** https://doi.org/10.3133/twri03b7

## Paso 1. Definición de parámetros del problema.

### Parámetros para la discretización del espacio.

|Parameter | Value| Units | Variable |
|---:|:---:|:---:|:---|
|Length of system (rows) |12.0| cm | `mesh.row_length` |
|Number of layers |1| | `mesh.nlay` |
|Number of rows |1| | `mesh.nrow` |
|Number of columns |120| | `mesh.ncol` |
|Column width |0.1| cm | `mesh.delr` |
|Row width |0.1| cm | `mesh.delc`|
|Top of the model |1.0| cm | `mesh.top`|
|Layer bottom elevation (cm) |0| cm | `mesh.bottom` |

In [ ]:
mesh = xmf6.MeshDis(
    nrow = 1,    # Number of rows
    ncol = 120,  # Number of columns
    nlay = 1,    # Number of layers
    row_length = 12.0,    # Length of system ($cm$)
    column_length = 0.1,  # Length of system ($cm$)
    top = 1.0,   # Top of the model ($cm$)
    bottom = 0,  # Layer bottom elevation ($cm$)
)
xmf6.nice_print(mesh.get_dict(), 'Space discretization')

### Parámetros para la discretización del tiempo.

* La simulación consta de un período de estrés que tiene una duración de 120 $s$.
* El período de estrés se divide en 240 pasos de tiempo del mismo tamaño.
  
|Parameter | Value| Units | Variable |
|---:|:---:|:---:|:---|
|Number of stress periods |1| | `tm_par['nper']` |
|Total time |120| s | `tm_par['total_time']` |
|Number of time steps| 240 | | `tm_par['nstp']`|
|Multiplier | 1 | | `tm_par['tsmult']`|

In [ ]:
tdis = xmf6.TDis(
    perioddata = ((120, 240, 1.0),) # PERLEN, NSTP, TSMULT
)
xmf6.nice_print(tdis, 'Time discretization')

### Parámetros físicos.

* Al utilizar un valor de porosidad uniforme de 0.1, se obtiene un valor de velocidad de 0.1 $cm/s$ como resultado de la inyección de agua a una velocidad de 0.001 $cm^3/s$ en la primera celda.
* A la última celda se le asigna una carga constante con un valor de cero, aunque este valor no es importante ya que las celdas están marcadas como confinadas.
* A la concentración del agua inyectada se le asigna un valor de 1.0, y cualquier agua que salga a través de la celda de carga constante sale con la concentración simulada del agua en esa última celda.
* La advección se resuelve utilizando el esquema TVD para reducir la dispersión numérica.
  
|Parameter | Value| Units | Variable |
|---:|:---:|:---:|:---|
|Specific discharge |0.1| cm s$^{-1}$ | `ph_par['specific_discharge']` |
|Hydraulic conductivity |0.01| cm s$^{-1}$ | `ph_par['hydraulic_conductivity']` |
|Source concentration |1.0| unitless | `ph_par['source_concentration']` |
|Porosity | 0.1 | unitless |  `ph_par['porosity']` |
|Initial Concentration | 0.0 | unitless | `ph_par['initial_concentration']` |
|Longitudinal Dispersivity | 0.1 |  | `ph_par['longitudinal_dispersivity']` |
|Retardation Factor | 1.0 |  | `ph_par['retardation_factor']` |
|Decay Rate | 0.0 |  | `ph_par['decay_rate']` |
|Dispersion coefficient | | | `ph_par["dispersion_coefficient"]` |

$$
\text{Dispersion Coefficient} = \dfrac{\text{Longitudinal Dispersivity} \times \text{Specific Discharge}}{\text{Retardation Factor}}
$$

In [ ]:
ml_units = {
    "time": "seconds",
    "length": "centimeters"
}
xmf6.nice_print(ml_units, 'Units')

ph_par = dict(
    specific_discharge = 0.1,  # Specific discharge ($cm s^{-1}$)
    hydraulic_conductivity = 0.01,  # Hydraulic conductivity ($cm s^{-1}$)
    source_concentration = 1.0,  # Source concentration (unitless)
    porosity = 0.1,  # Porosity of mobile domain (unitless)
    initial_concentration = 0.0,  # Initial concentration (unitless)
    longitudinal_dispersivity = 0.1,
    retardation_factor = 1.0,
    decay_rate =  0.0
)
ph_par["dispersion_coefficient"] = ph_par["longitudinal_dispersivity"] * ph_par["specific_discharge"] / ph_par["retardation_factor"]

xmf6.nice_print(ph_par, 'Physical parameters')

## Paso 2. MODFLOW6 environment y salida.

In [ ]:
# ----- Definición de Parámetros -----
os_par = dict(
    ws = os.getcwd() + '/output', # Ruta de donde estamos actualmente
    mf6_exe = '../../mf6/bin/mf6', # Ejecutable
    flow_name = 'flow', # Nombre de la simulación para flujo
    tran_name = 'transport' # Nombre de la simulación para transporte
)
xmf6.nice_print(os_par, 'MODFLOW 6 environment')

oc_par = dict(
    head_file = f"{os_par['flow_name']}.hds",
    fbudget_file = f"{os_par['flow_name']}.bud",
    concentration_file=f"{os_par['tran_name']}.ucn",
    tbudget_file = f"{os_par['tran_name']}.bud",
)
xmf6.nice_print(oc_par, 'Output files')

## Paso 3. Solución del flujo (Modelo GWF)

### Resultados del flujo

In [ ]:
from flow_1D import build_gwf_1D, plot_flow_1D
sim_f, gwf = build_gwf_1D(mesh, tdis, ph_par, ml_units, os_par, oc_par)
sim_f.write_simulation(silent=True)
sim_f.run_simulation(silent=True)
plot_flow_1D(gwf, mesh, os_par, oc_par)

### Obtención de arreglos de Numpy para la carga y  la velocidad.

In [ ]:
# Obtenemos los resultados de la carga hidráulica
head = flopy.utils.HeadFile(
    os.path.join(os_par['ws'], 
                 oc_par['head_file'])).get_data()
# Obtenemos los resultados del BUDGET
bud  = flopy.utils.CellBudgetFile(
    os.path.join(os_par['ws'], 
                 oc_par['fbudget_file']),
    precision='double'
)
# Obtenemos las velocidades
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

In [ ]:
print(head.shape, '\n', head)

In [ ]:
print(qx.shape, '\n', qx)

## Paso 4. Solución de transporte (Modelo GWT)

### Paso 4.1 Objeto de simulación

In [ ]:
sim_t = flopy.mf6.MFSimulation(
    sim_name=os_par["tran_name"], 
    sim_ws=os_par["ws"], 
    exe_name=os_par["mf6_exe"]
)
print(sim_t)

### Paso 4.2. Componente para el paso de tiempo

In [ ]:
flopy.mf6.ModflowTdis(
    sim_t, 
    nper=tdis.nper(), 
    perioddata=tdis.perioddata(), 
    time_units=ml_units["time"]
)

### Paso 4.3. IMS object (solution calculation)

In [ ]:
ims = flopy.mf6.ModflowIms(
    sim_t, 
    linear_acceleration="bicgstab"
)
print(ims)

### Paso 4.4. GWT model object (transport)

In [ ]:
gwt = flopy.mf6.ModflowGwt(
    sim_t, 
    modelname=os_par["tran_name"], 
    save_flows=True
)
print(gwt)

### Paso 4.5. Space discretization object

In [ ]:
dis = flopy.mf6.ModflowGwtdis(
    gwt,
    length_units=ml_units["length"],
    nlay=mesh.nlay,
    nrow=mesh.nrow,
    ncol=mesh.ncol,
    delr=mesh.delr,
    delc=mesh.delc,
    top=mesh.top,
    botm=mesh.bottom,
)

print(dis)

### Paso 4.6. Initial conditions object

In [ ]:
ic = flopy.mf6.ModflowGwtic(
    gwt, 
    strt=0
)
print(ic)

### Paso 4.7.

In [ ]:
mst = flopy.mf6.ModflowGwtmst(
    gwt,
    porosity=ph_par["porosity"],
    **get_sorption_dict(ph_par["retardation_factor"]),
    **get_decay_dict(ph_par["decay_rate"], 
                     ph_par["retardation_factor"] > 1.0),
)

print(mst)

### Paso 4.8. Advection

In [ ]:
adv = flopy.mf6.ModflowGwtadv(
    gwt, 
    scheme="TVD"
)
print(adv)

### Paso 4.9. 

In [ ]:
dsp = flopy.mf6.ModflowGwtdsp(
    gwt,
    xt3d_off=True,
    alh=ph_par["longitudinal_dispersivity"],
    ath1=ph_par["longitudinal_dispersivity"],
)

print(dsp)

### Paso 4.10. 

In [ ]:
pd = [ 
    ("GWFHEAD", oc_par["head_file"], None),
    ("GWFBUDGET", oc_par["fbudget_file"], None),
]
    
fmi = flopy.mf6.ModflowGwtfmi(
    gwt, 
    packagedata=pd
)
print(fmi)

### Paso 4.11. 

In [ ]:
sourcerecarray = [["WEL-1", "AUX", "CONCENTRATION"]]
ssm = flopy.mf6.ModflowGwtssm(
    gwt, 
    sources=sourcerecarray
)
print(ssm)

### Paso 4.12. Observation locations

In [ ]:
obs_data = {
    "transporte.obs.csv": [
        ("X005", "CONCENTRATION", (0, 0, 0)),
        ("X405", "CONCENTRATION", (0, 0, 40)),
        ("X1105", "CONCENTRATION", (0, 0, 110)),
    ],
}

obs_package = flopy.mf6.ModflowUtlobs(
    gwt, 
    digits=10, 
    print_input=True, 
    continuous=obs_data
)

print(obs_package)

### Paso 4.13. Output object

In [ ]:
oc = flopy.mf6.ModflowGwtoc(
    gwt,
    budget_filerecord=oc_par["tbudget_file"],
    concentration_filerecord=oc_par["concentration_file"],
    saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "LAST")],
    printrecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],
)

print(oc)

### Paso 4.14. Write Input files.

In [ ]:
sim_t.write_simulation()

### Paso 4.15. Run simulation

In [ ]:
sim_t.run_simulation()

### Paso 4.16. Postprocessing

In [ ]:
sim_t.model_names

In [ ]:
mf6gwt_ra = sim_t.get_model("transport").obs.output.obs().data
ucnobj_mf6 = sim_t.transport.output.concentration()

In [ ]:
print(type(mf6gwt_ra), type(ucnobj_mf6))

In [ ]:
print(mf6gwt_ra.shape)

In [ ]:
print(mf6gwt_ra)

In [ ]:
simtimes = mf6gwt_ra["totim"]
simtimes.shape
print(type(simtimes), simtimes)

In [ ]:
obsnames = ["X005", "X405", "X1105"]

In [ ]:
# Funciones para cálculo de la solución analítica
from wexler1_test import sol_analytical_t

with styles.USGSPlot():
    plt.rcParams['font.family'] = 'DeJavu Sans'

    fig, axs = plt.subplots(2, 1, figsize=(5,6), tight_layout=True)

    iskip = 5

    
    atimes = np.arange(0, tdis.total_time(), 0.1)
    
    for i, x in enumerate([0.05, 4.05, 11.05]):
        a1, idx_filter = sol_analytical_t(i, x, atimes,mesh, ph_par) 
                
        axs[0].plot(atimes[idx_filter], a1[idx_filter], color="k", label="ANALYTICAL")

        axs[0].plot(simtimes[::iskip], mf6gwt_ra[obsnames[i]][::iskip],
                    marker="o", ls="none", mec="blue", mfc="none", markersize="4",
                    label="MODFLOW 6")
        axs[0].set_ylim(-0.05, 1.2)
        axs[0].set_xlim(0, 120)
        axs[0].set_xlabel("Time (seconds)")
        axs[0].set_ylabel("Normalized Concentration (unitless)")
        
    ctimes = [6.0, 60.0, 120.0]

    
    x, _, _ = mesh.get_coords()
    for i, t in enumerate(ctimes):
        a1, idx_filter = sol_analytical_t(i, x, t, mesh, ph_par, False)
        
        axs[1].plot(x, a1, color="k", label="ANALYTICAL")
        simconc = ucnobj_mf6.get_data(totim=t).flatten()
        axs[1].plot(x[::iskip], simconc[::iskip],
                    marker="o", ls="none", mec="blue", mfc="none", markersize="4",
                    label="MODFLOW 6")
        axs[1].set_ylim(0, 1.1)
        axs[1].set_xlim(0, 12)
        axs[1].set_xlabel("Distance (cm)")
        axs[1].set_ylabel("Normalized Concentration (unitless)")
    
    plt.show()